In [4]:
import os
import re
from osgeo import gdal

from tqdm.auto import tqdm
import richdem as rd
import rasterio as rio
from utils import access_raster

In [2]:
local_path = "/home/abetatos/Escritorio/TFM_Dataset/0.Dataset/"
unitato_epsg = "/home/abetatos/Escritorio/TFM_Dataset/1.Dataset_epsg/"

In [3]:
EPSG = "EPSG:2056" # Switzerland area

# HS

In [7]:
input_path_HS = os.path.join(local_path, "HS")
output_path_HS = os.path.join(unitato_epsg, "HS")
os.makedirs(output_path_HS, exist_ok=True)

In [8]:
files = [f for f in os.listdir(input_path_HS) if re.search(".tiff?$", f)]
files

['snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser.tif',
 'HS_Davos_20211604_2m_epsg2056.tiff',
 'snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser.tif',
 'HS_Davos_20170316_2m_epsg2056.tiff',
 'HS_Laucherenalp_20220224_2m_epsg2056.tif',
 'snowmap_2010_ads_80_extent_maskiert_nosnow0_noausreisser.tif',
 'HS_c_20220223_1m_epsg2056.tif',
 'snowmap_20160420_ads_100_extent_maskiert_nosnow0_noausreisser.tif',
 'snowmap_2014_ads_100_extent_maskiert_nosnow0_noausreisser.tif',
 'snowmap_2013_ads_80_extent_maskiert_nosnow0_noausreisser.tif',
 'HS_Davos_20180411_2m_epsg2056.tiff',
 'HS_Laucherenalp_20220511_1m_epsg2056.tif',
 'HS_Saflischpass_20220512_1m_epsg2056.tif',
 'HS_Davos_20200406_2m_epsg2056.tiff',
 'HS_Davos_20190316_2m_epsg2056.tif',
 'snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser.tif',
 'snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser.tif']

In [9]:
for f in files: 
    try: 
        access_raster(os.path.join(input_path_HS, f))
    except: 
        print("failed", f)
        continue
    print(f)
    gdal.Warp(os.path.join(output_path_HS, f).replace(".tiff", ".tif"), os.path.join(input_path_HS, f), dstSRS=EPSG)

snowmap_20160309_ads_100_extent_maskiert_nosnow0_noausreisser.tif
HS_Davos_20211604_2m_epsg2056.tiff
snowmap_2012_ads_80_extent_maskiert_nosnow0_noausreisser.tif
HS_Davos_20170316_2m_epsg2056.tiff
HS_Laucherenalp_20220224_2m_epsg2056.tif
snowmap_2010_ads_80_extent_maskiert_nosnow0_noausreisser.tif
HS_c_20220223_1m_epsg2056.tif
snowmap_20160420_ads_100_extent_maskiert_nosnow0_noausreisser.tif
snowmap_2014_ads_100_extent_maskiert_nosnow0_noausreisser.tif
snowmap_2013_ads_80_extent_maskiert_nosnow0_noausreisser.tif
HS_Davos_20180411_2m_epsg2056.tiff
HS_Laucherenalp_20220511_1m_epsg2056.tif
HS_Saflischpass_20220512_1m_epsg2056.tif
HS_Davos_20200406_2m_epsg2056.tiff
failed HS_Davos_20190316_2m_epsg2056.tif
snowmap_20160126_ads_100_extent_maskiert_nosnow0_noausreisser.tif
snowmap_2015_ads_100_extent_maskiert_nosnow0_noausreisser.tif


# DEM

In [13]:
get_file = lambda location: rf"{local_path}DEM/{location}/all/mosaic_output.tif"
output_path = rf"{unitato_epsg}DEM"
os.makedirs(output_path, exist_ok=True)
for location in ['Davos', 'Valais']: 
    print(location)
    input_file = get_file(location)
    output_file = os.path.join(output_path, f"{location}.tif")
    gdal.Warp(output_file, input_file, dstSRS=EPSG)

Davos
Valais


# SCE

In [5]:
input_path_SCE = f"{local_path}SCE_Switz"
output_path_SCE = rf"{unitato_epsg}SCE"
os.makedirs(output_path_SCE, exist_ok=True)

files = [f for f in os.listdir(input_path_SCE) if re.search(".tiff?$", f)]

for f in tqdm(files): 
    try: 
        array = access_raster(os.path.join(input_path_SCE, f))['array']
    except: 
        print("failed", f)
        continue
    gdal.Warp(os.path.join(output_path_SCE, f).replace(".tiff", ".tif"), os.path.join(input_path_SCE, f), dstSRS=EPSG)

  0%|          | 0/146 [00:00<?, ?it/s]

Counter({200: 135497766, 190: 87621058, 10: 49477161, 100: 48970980, 140: 29291267, 160: 27250087, 130: 26077818, 170: 16694723, 180: 15263985, 150: 14246798, 40: 8868260, 110: 7961717, 30: 4227657, 20: 3856085, 0: 3197137, 120: 2821633, 80: 1381246, 70: 1172383, 90: 1076142, 60: 679120, 50: 531656})


# Aspect

Obtaining it from DEM

In [24]:
get_file = lambda location: f"{local_path}DEM/{location}/all/mosaic_output.tif"
get_aspect_file = lambda location: rf"{local_path}Aspect/{location}.tif"
output_path = rf"{unitato_epsg}Aspect"

os.makedirs(f"{local_path}Aspect", exist_ok=True)
os.makedirs(output_path, exist_ok=True)

for location in ['Davos', 'Valais']: 
    print(location)
    input_file = get_file(location)
    aspect_file = get_aspect_file(location)
    
    file = rd.LoadGDAL(input_file)
    aspect = rd.TerrainAttribute(file, attrib='aspect')
    
    with rio.open(input_file) as src: 
        profile = src.profile

    with rio.open(aspect_file, "w", **profile) as src: 
        src.write(aspect, 1)
        
    output_file = os.path.join(output_path, f"{location}.tif")
    gdal.Warp(output_file, aspect_file, dstSRS=EPSG)

Davos



A Aspect attribute calculation
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 7.18453                         ] (8% - 82.5s - 1 threads))


Valais



A Aspect attribute calculation
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 8.09421                         ] (2% - 396.4s - 1 threads)


# Slope

Obtaining it from DEM

In [26]:
get_file = lambda location: f"{local_path}DEM/{location}/all/mosaic_output.tif"
get_slope_file = lambda location: rf"{local_path}Slope/{location}.tif"

os.makedirs(f"{local_path}Slope", exist_ok=True)
output_path = rf"{unitato_epsg}Slope"
os.makedirs(output_path, exist_ok=True)

for location in ['Davos', 'Valais']: 
    print(location)
    input_file = get_file(location)
    slope_file = get_slope_file(location)
    
    file = rd.LoadGDAL(input_file)
    slope = rd.TerrainAttribute(file, attrib='slope_riserun')
    
    with rio.open(input_file) as src: 
        profile = src.profile

    with rio.open(slope_file, "w", **profile) as src: 
        src.write(slope, 1)
        
    output_file = os.path.join(output_path, f"{location}.tif")
    gdal.Warp(output_file, slope_file, dstSRS=EPSG)

Davos



A Slope calculation (rise/run)
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 2.99119                         ] (8% - 34.3s - 1 threads))


Valais



A Slope calculation (rise/run)
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 3.34929                         ] (2% - 164.0s - 1 threads)


# TPI

In [5]:
from whitebox import WhiteboxTools

wbt = WhiteboxTools()

In [6]:
get_file = lambda location: f"{local_path}DEM/{location}/all/mosaic_output.tif"
get_slope_file = lambda location: rf"{local_path}TPI/{location}.tif"
os.makedirs(f"{local_path}TPI", exist_ok=True)
output_path = rf"{unitato_epsg}TPI"
os.makedirs(output_path, exist_ok=True)

# for filter_size in [11, 22, 50, 7, 5, 9]: 
for filter_size in [11]:
    get_tpi_file = lambda location: rf"{local_path}TPI/{location}_{filter_size}.tif"
    for location in ['Davos', 'Valais']: 
        print([filter_size, location])
        input_file = get_file(location)
        tpi_file = get_tpi_file(location)
        
        wbt.relative_topographic_position(input_file, tpi_file, filterx=filter_size, filtery=filter_size)
        
        output_file = os.path.join(output_path, f"{location}_{filter_size}.tif")
        gdal.Warp(output_file, tpi_file, dstSRS=EPSG)

[11, 'Davos']
./whitebox_tools --run="RelativeTopographicPosition" --dem='/home/abetatos/Escritorio/TFM_Dataset/Dataset/DEM/Davos/all/mosaic_output.tif' --output='/home/abetatos/Escritorio/TFM_Dataset/Dataset/TPI/Davos_11.tif' --filterx=11 --filtery=11 -v --compress_rasters=False

******************************************
* Welcome to RelativeTopographicPosition *
* Powered by WhiteboxTools               *
* www.whiteboxgeo.com                    *
******************************************
Reading data...
Performing analysis: 0%
Performing analysis: 1%
Performing analysis: 2%
Performing analysis: 3%
Performing analysis: 4%
Performing analysis: 5%
Performing analysis: 6%
Performing analysis: 7%
Performing analysis: 8%
Performing analysis: 9%
Performing analysis: 10%
Performing analysis: 11%
Performing analysis: 12%
Performing analysis: 13%
Performing analysis: 14%
Performing analysis: 15%
Performing analysis: 16%
Performing analysis: 17%
Performing analysis: 18%
Performing analysis: 1

# TPI WGW

In [5]:
from epsg_utils import tpi
import time 
time.sleep(1)

In [6]:
get_file = lambda location: f"{local_path}DEM/{location}/all/mosaic_output.tif"
get_slope_file = lambda location: rf"{local_path}TPIWGW/{location}.tif"
os.makedirs(f"{local_path}TPIWGW", exist_ok=True)
output_path = rf"{unitato_epsg}TPIWGW"
os.makedirs(output_path, exist_ok=True)


# for filter_size in [11, 22, 50, 7, 5, 9]: 
for filter_size in [11]:
    get_tpi_file = lambda location: rf"{local_path}TPIWGW/{location}_{filter_size}.tif"
    for location in ['Davos', 'Valais']:
        print([filter_size, location])
        input_file = get_file(location)
        tpi_file = get_tpi_file(location)
        
        tpi(elevation_model=input_file,r=filter_size,output_model=tpi_file)
        print("tpi obtained")
        time.sleep(60)
        output_file = os.path.join(output_path, f"{location}_{filter_size}.tif")
        gdal.Warp(output_file, tpi_file, dstSRS=EPSG)


[11, 'Valais']
tpi obtained
